In [1]:
import cv2 as cv
import numpy as np


In [2]:
import mediapipe as mp

In [3]:
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils



In [4]:
wave_path = "./wave"
import os
files = os.listdir(wave_path)


In [13]:

counter = []
with mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    
    for file in files:
        cap = cv.VideoCapture(os.path.join(wave_path , file))
        count=0
        
        while cap.isOpened():
            ret, frame = cap.read()
            count+=1
            results = pose.process(cv.cvtColor(frame, cv.COLOR_BGR2RGB))
            if not ret or count >=60:
                break
            print(count    , end = "\r")
            mp_drawing.draw_landmarks(
                frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS
            )
            cv.imshow("frame", frame)
            if cv.waitKey(10) & 0xff == ord("q"):
                break
        counter.append(count)

In [1]:
import pandas as pd


In [2]:
def append_csv(par_folder, files,seq_frames, target):
    import os
    for file in files:
        df = pd.read_csv(os.path.join(par_folder, file))
        df.drop("count", axis=1, inplace=True)
        target.append(df["class"].iloc[0])

        df.drop("class", axis=1, inplace=True)
        values = df.values
        if len(values)!=36:
            print(file+ " not long enough " + str(len(values)))
        seq_frames.append(values)
        
        

In [3]:
seq_frames = []
target = []
par_folder = "./notCsv"
files = os.listdir(par_folder)
append_csv(par_folder, files, seq_frames, target)
par_folder = "./waveCsv"
files = os.listdir(par_folder)
append_csv(par_folder, files, seq_frames, target)


data_WIS_video_walk_lena_walk.csv not long enough 35
data_WIS_video_bend_ira_bend.csv not long enough 35
data_WIS_video_jack_eli_jack.csv not long enough 35
data_WIS_video_bend_daria_bend.csv not long enough 35
data_WIS_video_skip_shahar_skip.csv not long enough 35
data_WIS_video_run_lyova_run.csv not long enough 35
data_WIS_video_wave1_lena_wave1.csv not long enough 35


In [4]:
len(seq_frames), len(target)

(18, 18)

In [5]:
import numpy as np
X_train = []
for seq in seq_frames:
    if seq.shape[0] < 36:
        # print(seq[-1, :], seq[-1, :].shape)
        X_train.append(np.vstack([seq, seq[-1, :]]))
        print(X_train[-1].shape[0])
    else:
        X_train.append(np.array(seq))

36
36
36
36
36
36
36


In [6]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras.layers import Dense
from keras.layers import LSTM
from keras.models import Sequential
from keras.models import load_model
from keras.callbacks import ModelCheckpoint
from tensorflow.keras import losses
from keras.layers import TimeDistributed


In [7]:
model = Sequential()
model.add(LSTM(128, input_shape=(36, 33*4),return_sequences=True, activation="relu" ))
model.add(LSTM(64, activation='relu'))
# model.add(TimeDistributed(Dense(12, activation="relu")))
# model.add(keras.layers.Flatten())

model.add(Dense(1, activation="sigmoid"))


In [8]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 36, 128)           133632    
_________________________________________________________________
lstm_1 (LSTM)                (None, 64)                49408     
_________________________________________________________________
dense (Dense)                (None, 1)                 65        
Total params: 183,105
Trainable params: 183,105
Non-trainable params: 0
_________________________________________________________________


In [9]:
X_train[0].shape, target

((36, 132),
 ['not',
  'not',
  'not',
  'not',
  'not',
  'not',
  'not',
  'not',
  'not',
  'wave',
  'wave',
  'wave',
  'wave',
  'wave',
  'wave',
  'wave',
  'wave',
  'wave'])

In [10]:
X_train = np.array(X_train)
t = [1 if x == 'wave' else 0 for x in target]
target = np.array(t)


print(X_train.shape)
type(seq_frames[0])
print(t)


(18, 36, 132)
[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [11]:
from sklearn.model_selection import train_test_split

x_train, validation_X,  y_train,validation_y = train_test_split(X_train, target, test_size=0.2)

In [12]:
validation_y

array([1, 1, 0, 1])

In [14]:
adam = keras.optimizers.Adam(learning_rate=0.001)
chk = ModelCheckpoint("best_model2.pkl", monitor="accuracy", save_best_only=True, mode="max", verbose=1)
model.compile(loss="binary_crossentropy", optimizer=adam, metrics=["accuracy"])
model.fit(x_train,y_train, epochs=100, batch_size=4, callbacks=[chk], validation_data=(validation_X,validation_y))


Epoch 1/100
4/4 [==============================] - 3s 271ms/step - loss: 0.7424 - accuracy: 0.5000 - val_loss: 0.7104 - val_accuracy: 0.2500

Epoch 00001: accuracy improved from -inf to 0.50000, saving model to best_model2.pkl


INFO:tensorflow:Assets written to: best_model2.pkl/assets


INFO:tensorflow:Assets written to: best_model2.pkl/assets


Epoch 2/100
4/4 [==============================] - 1s 196ms/step - loss: 0.6435 - accuracy: 0.5714 - val_loss: 0.8787 - val_accuracy: 0.2500

Epoch 00002: accuracy improved from 0.50000 to 0.57143, saving model to best_model2.pkl


INFO:tensorflow:Assets written to: best_model2.pkl/assets


...

INFO:tensorflow:Assets written to: best_model2.pkl/assets


Epoch 7/100
4/4 [==============================] - 0s 114ms/step - loss: 0.4480 - accuracy: 0.7143 - val_loss: 0.3980 - val_accuracy: 1.0000

Epoch 00007: accuracy did not improve from 0.92857
Epoch 8/100
4/4 [==============================] - 0s 87ms/step - loss: 0.3715 - accuracy: 0.8571 - val_loss: 0.1711 - val_accuracy: 1.0000

Epoch 00008: accuracy did not improve from 0.92857
Epoch 9/100
4/4 [==============================] - 0s 76ms/step - loss: 37.4479 - accuracy: 0.7857 - val_loss: 0.0027 - val_accuracy: 1.0000

Epoch 00009: accuracy did not improve from 0.92857
Epoch 10/100
4/4 [==============================] - 1s 144ms/step - loss: 1.0315 - accuracy: 0.5714 - val_loss: 0.3296 - val_accuracy: 0.7500

Epoch 00010: accuracy did not improve from 0.92857
Epoch 11/100
4/4 [==============================] - 0s 89ms/step - loss: 0.9310 - accuracy: 0.5000 - val_loss: 7.1636 - val_accuracy: 0.2500

Epoch 00011: accuracy did not improve from 0.92857
Epoch 12/100
4/4 [=================

INFO:tensorflow:Assets written to: best_model2.pkl/assets


INFO:tensorflow:Assets written to: best_model2.pkl/assets


Epoch 22/100
4/4 [==============================] - 0s 77ms/step - loss: 0.1661 - accuracy: 1.0000 - val_loss: 0.0260 - val_accuracy: 1.0000

Epoch 00022: accuracy did not improve from 1.00000
Epoch 23/100
4/4 [==============================] - 0s 86ms/step - loss: 0.2140 - accuracy: 0.8571 - val_loss: 1.3863 - val_accuracy: 0.5000

Epoch 00023: accuracy did not improve from 1.00000
Epoch 24/100
4/4 [==============================] - 0s 72ms/step - loss: 0.2037 - accuracy: 0.9286 - val_loss: 0.0458 - val_accuracy: 1.0000

Epoch 00024: accuracy did not improve from 1.00000
Epoch 25/100
4/4 [==============================] - 0s 73ms/step - loss: 0.4058 - accuracy: 0.8571 - val_loss: 0.1127 - val_accuracy: 1.0000

Epoch 00025: accuracy did not improve from 1.00000
Epoch 26/100
4/4 [==============================] - 0s 73ms/step - loss: 0.3028 - accuracy: 0.8571 - val_loss: 0.3226 - val_accuracy: 1.0000

Epoch 00026: accuracy did not improve from 1.00000
Epoch 27/100
4/4 [=================

In [119]:
a = np.array([1,2,3,4])
b = np.array([[4,5,6,6]])
np.vstack([a, b])

array([[1, 2, 3, 4],
       [4, 5, 6, 6]])

In [2]:
import numpy as np

x = np.array([[1,2,3,3,4,5],
             [2,3,34,4,5,5]])
x.shape

(2, 6)

In [8]:
x=x.reshape(1,x.shape[0], x.shape[1])


In [9]:
x.shape

(1, 2, 6)

In [10]:
x

array([[[ 1,  2,  3,  3,  4,  5],
        [ 2,  3, 34,  4,  5,  5]]])

In [18]:
from keras.models import Model
from keras.layers import Input
from keras.layers import LSTM
from numpy import array
# define model
inputs1 = Input(shape=(3, 1))
lstm1, state_h, state_c = LSTM(1, return_state=True)(inputs1)
model = Model(inputs=inputs1, outputs=[lstm1, state_h, state_c])
# define input data
data = array([0.1, 0.2, 0.3]).reshape((1,3,1))
# make and show prediction
print(model.predict(data))

[array([[-0.10178033]], dtype=float32), array([[-0.10178033]], dtype=float32), array([[-0.2346895]], dtype=float32)]


In [22]:
inputs1 = Input(shape=(3, 1))
lstm1 = LSTM(1, return_sequences=True)(inputs1)
model = Model(inputs=inputs1, outputs=lstm1)
# define input data
data = array([0.1, 0.2, 0.3]).reshape((1,3,1))
# make and show prediction
print(model.predict(data))

[[[0.00284738]
  [0.00767078]
  [0.01395339]]]


In [30]:
length = 5
seq = array([i/float(length) for i in range(length)])
X = seq.reshape(1, length, 1)
y = seq.reshape(1, length, 1)
y

array([[[0. ],
        [0.2],
        [0.4],
        [0.6],
        [0.8]]])